In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 加载视频

In [2]:
cap = cv2.VideoCapture('./video/car.mp4')

In [4]:
# # 循环读取视频帧
# while True:
#     ret, frame = cap.read()
#     if ret == True:
#         cv2.imshow('video', frame)
        
#     key = cv2.waitKey(1)
#     # esc
#     if key == 27:
#         break
        
# # 释放资源
# cap.release()
# cv2.destroyAllWindows()

# 去除背景示例

In [ ]:
# cap = cv2.VideoCapture(0)
# bgs = cv2.bgsegm.createBackgroundSubtractorMOG()

# while True:
#     ret, frame = cap.read()
    
#     if ret == True:
#         fgmask = bgs.apply(frame)
#         cv2.imshow('video', fgmask)
#     key = cv2.waitKey(1)
#     if key == 27:
#         break
          
# cap.release()
# cv2.destoryAllWindows()

# 形态学识别车辆

In [ ]:
# cap = cv2.VideoCapture('./video/car.mp4')

# bgs = cv2.bgsegm.createBackgroundSubtractorMOG()

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# while True:
#     ret, frame = cap.read()
#     if ret == True:
#         # 把原始帧进行灰度化，然后去噪
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         # 去噪
#         blur = cv2.GaussianBlur(gray, (3, 3), 5)
#         fgmask = bgs.apply(blur)
        
#         # 腐蚀
#         erode = cv2.erode(fgmask, kernel)
#         # 膨胀
#         dilate = cv2.dilate(erode, kernel, iterations=2)
        
#         # 消除内部的小方块 - 闭运算
#         close = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)
        
#         cv2.imshow('video', close)
        
#     key = cv2.waitKey(10)
#     # esc
#     if key == 27:
#         break
        
# # 释放资源
# cap.release()
# cv2.destroyAllWindows()

# 查找轮廓

In [ ]:
cap = cv2.VideoCapture('./video/car.mp4')

bgs = cv2.bgsegm.createBackgroundSubtractorMOG()

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

min_w, min_h = 100, 90
line_high = 600

# 偏移量
offset = 7

cars = []
cars_count = 0

# 计算中心点
def center(x, y, w, h):
    return int(w / 2 + x), int(h / 2 + y)

while True:
    ret, frame = cap.read()
    if ret == True:
        # 把原始帧进行灰度化，然后去噪
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 去噪
        blur = cv2.GaussianBlur(gray, (3, 3), 5)
        fgmask = bgs.apply(blur)
        
        # 腐蚀
        erode = cv2.erode(fgmask, kernel)
        # 膨胀
        dilate = cv2.dilate(erode, kernel, iterations=2)
        
        # 消除内部的小方块 - 闭运算
        close = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)
        
        # 查找轮廓
        contours, _ = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # 画出检测线
        cv2.line(frame, (10, line_high), (1200, line_high), (0, 255, 0), 3)
        
        # 画出所有检测出来的轮廓
        for contour in contours:
            # 最大外接矩形
            (x, y, w, h) = cv2.boundingRect(contour)
            
            # 通过外接矩形的宽高，过滤多余过小矩形
            is_vaild = (w >= min_w) & (h >= min_h)
            if not is_vaild:
                continue
                
            # 要求坐标点都是整数
            cv2.rectangle(frame, (int(x), int(y)), (int(x+w), int(y+h)), (0, 0, 255), 2)
            
            # 把车抽象成一点，即外接矩形的中心点
            cpoint = center(x, y, w, h)
            cars.append(cpoint)
            cv2.circle(frame, (cpoint), 5, (0, 0, 255), -1)
            
            # 判断汽车是否过线
            for (x, y) in cars:
                if y > (line_high - offset) and y < (line_high + offset):
                    # 有效
                    # count
                    cars_count += 1
                    cars.remove((x, y))
        
        cv2.putText(frame, "Vehicle Cpount:" + str(cars_count), (500, 60), \
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 5)
        cv2.imshow('video', frame)
        
    key = cv2.waitKey(10)
    # esc
    if key == 27:
        break
        
# 释放资源
cap.release()
cv2.destroyAllWindows()